Creating tables in SQL
---------------------

Before we actually get into basic SQL queries (**asking questions _of_ data in tables**), we'll look at some of the basics about how to **create** tables.

In [1]:
%load_ext sql
%sql sqlite://

We will start by creating a SQL database on Books and Authors. Let's first create a table named `Author` with three attributes:
> * `authorid`
> * `firstname`
> * `lastname`

In [5]:
%%sql
CREATE TABLE Author(
    authorid INTEGER PRIMARY KEY,    
    firstname CHAR(20),
    lastname CHAR(30)
);

 * sqlite://
Done.


[]

Let's see how to view the **schema** of existing tables. There are several ways, including but not limited to:
* DESCRIBE tablename
* SHOW CREATE TABLE tablename
* SHOW COLUMNS tablename

Unfortunately, support for these varies widely between DBMSs, and is also limited by our IPython interface (for example sqlite, which we are using, does not support the above; it does have a `.schema tablename` command, however this doesn't work in IPython notebooks...)

One that does work for us here though is:

In [6]:
#Metadata in SQLite can be obtained using the PRAGMA command
%sql PRAGMA table_info(Author);

 * sqlite://
Done.


cid,name,type,notnull,dflt_value,pk
0,authorid,INTEGER,0,None,1
1,firstname,CHAR(20),0,None,0
2,lastname,CHAR(30),0,None,0


And, we can get the exact statement used to create the table as follows:

In [7]:
%sql SELECT sql FROM sqlite_master WHERE name = 'Author';

 * sqlite://
Done.


sql
"CREATE TABLE Author( authorid INTEGER PRIMARY KEY, firstname CHAR(20), lastname CHAR(30))"


To insert a new tuple in a table, we use the SQL command INSERT.

In [8]:
%sql INSERT INTO Author VALUES(1,'Dan', 'Brown');

 * sqlite://
1 rows affected.


[]

Alternatively, we can also insert a tuple by specifying the corresponding attributes.

In [9]:
%sql INSERT INTO Author(authorid, lastname, firstname) VALUES(2, 'Clarke', 'Arthur');

 * sqlite://
1 rows affected.


[]

To check that the tuples has indeed been added to the table, we can ask:

In [10]:
%%sql 
SELECT *
FROM Author;

 * sqlite://
Done.


authorid,firstname,lastname
1,Dan,Brown
2,Arthur,Clarke


The attribute `authorid` is a *primary key*. This means that it is not possible to add in the table another author with the same value. For example, see what happens when we attempt to violate the constraint.

In [11]:
%sql INSERT INTO Author VALUES('01', 'Jules', 'Verne');

 * sqlite://


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: Author.authorid
[SQL: INSERT INTO Author VALUES('01', 'Jules' , 'Verne' );]
(Background on this error at: http://sqlalche.me/e/gkpj)

Foreign Keys
------------------

To make SQLite enforce foreign key constraints, we need to run the following command

In [12]:
%sql PRAGMA foreign_keys = ON;

 * sqlite://
Done.


[]

In [16]:
%%sql
CREATE TABLE Book(
  bookid INTEGER PRIMARY KEY,
  title TEXT,
  authorid INTEGER,
  FOREIGN KEY (authorid) REFERENCES Author(authorid)
);

 * sqlite://
Done.


[]

In [17]:
%sql INSERT INTO Book VALUES(5634, 'Da Vinvi Code', 01);

 * sqlite://
1 rows affected.


[]

In [19]:
%sql INSERT INTO Author VALUES('47', 'Michael', 'Jordan');

 * sqlite://
1 rows affected.


[]

In [20]:
%sql select * from author

 * sqlite://
Done.


authorid,firstname,lastname
1,Dan,Brown
2,Arthur,Clarke
47,Michael,Jordan


In [22]:
%sql INSERT INTO Book VALUES(5671, 'Starting Game', 002);

 * sqlite://
1 rows affected.


[]

In [23]:
%sql select * from book

 * sqlite://
Done.


bookid,title,authorid
5634,Da Vinvi Code,1
5671,Starting Game,2


If however we try to insert a tuple that has an `authorid` not in the table `Author`, then it fails.

In [25]:
%sql INSERT INTO Book VALUES(5647, 'Da Vinvi Code', 004);

 * sqlite://


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO Book VALUES(5647, 'Da Vinvi Code' , 004);]
(Background on this error at: http://sqlalche.me/e/gkpj)

In [26]:
%sql INSERT INTO Book VALUES(5648, 'Great Game', 0047);

 * sqlite://
1 rows affected.


[]

In [28]:
%sql select * from book

 * sqlite://
Done.


bookid,title,authorid
5634,Da Vinvi Code,1
5648,Great Game,47
5671,Starting Game,2


In [30]:
%sql DELETE FROM Book WHERE bookid = 5648 ;

 * sqlite://
1 rows affected.


[]

In [31]:
%sql select * from book

 * sqlite://
Done.


bookid,title,authorid
5634,Da Vinvi Code,1
5671,Starting Game,2


The same problem will occur if we try to delete from `Author` a tuple that is referred to from the table `Book`.

In [32]:
%sql DELETE FROM Author WHERE authorid = 1 ;

 * sqlite://


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: DELETE FROM Author WHERE authorid = 1 ;]
(Background on this error at: http://sqlalche.me/e/gkpj)

Or if we try to update the value of the tuple.

In [33]:
%%sql
UPDATE Author
SET authorid = 4
WHERE authorid = 1 ;

 * sqlite://


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: UPDATE Author SET authorid = 4
WHERE authorid = 1 ;]
(Background on this error at: http://sqlalche.me/e/gkpj)

The *default* behavior of SQL is to reject actions (insertions, deletions, updates) that violate the foreign key constraints. We can change this behavior by changing the table definition.

In [ ]:
%%sql
DROP Table Book;
CREATE TABLE Book(
  bookid INTEGER PRIMARY KEY,
  title TEXT,
  authorid INTEGER,
  FOREIGN KEY (authorid) REFERENCES Author(authorid)
    ON UPDATE CASCADE
    ON DELETE CASCADE
);

INSERT INTO Book VALUES(5634, 'Da Vinvi Code', 001);

Now let's try again!

In [ ]:
%%sql 
UPDATE Author
SET authorid = 4
WHERE authorid = 1 ;

In [ ]:
%sql SELECT * from Book ;